#### 1. Load Packages

Notes on Key Changes
Rolling Covariance (get_rolling_cov): We now use risk_models.exp_cov(sub_df, span=window) every day in regime_based_bl_backtest_flatprior().

Fallback in max_sharpe: A new EfficientFrontier(...) is instantiated as ef2 if ef.max_sharpe(...) fails.

Risk Aversion is set to 2.5 in the BlackLittermanModel(...).

Absolute Views remain: absolute_views=dict(zip(assets, view_vector)).

Transaction Cost is 0.0007 = 7 bps (unchanged usage).

One-Day Delay: On day t, we use states_test[t-1] to build a new portfolio, applying it at the end of day t. Meanwhile, the day t-1 returns are realized with the old weights.

Fallback Return = 0.0 for an unseen regime. also fallback function is equal weight rather that minimum volatility.

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import wilcoxon
from joblib import Parallel, delayed
import multiprocessing

# Hidden Markov Model utilities
from hmmlearn.hmm import GaussianHMM
from sklearn.cluster import KMeans

# PyPortfolioOpt
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models, expected_returns
from pypfopt.black_litterman import BlackLittermanModel

# Sparse Jump Model utilities
from jumpmodels.sparse_jump import SparseJumpModel
from jumpmodels.preprocess import StandardScalerPD, DataClipperStd

#### 2. Data simulation

In [6]:
ASSETS = ["Value", "Growth", "LowVol", "Size", "Momentum", "Quality"]
N_ASSETS = len(ASSETS)

def simulate_1state_data(num_days, seed=None):
    np_rng = np.random.default_rng(seed)
    SIM_MEAN_1STATE = 0.000461
    SIM_SIG_1STATE  = 0.008388

    corr = np.full((N_ASSETS, N_ASSETS), 0.185)
    np.fill_diagonal(corr, 1.0)
    cov = np.outer(np.full(N_ASSETS, SIM_SIG_1STATE),
                   np.full(N_ASSETS, SIM_SIG_1STATE)) * corr

    rets = np_rng.multivariate_normal(
        mean=np.full(N_ASSETS, SIM_MEAN_1STATE),
        cov=cov,
        size=num_days
    )
    return pd.DataFrame(rets, columns=ASSETS)


def simulate_2state_data(num_days, seed=None):
    np_rng = np.random.default_rng(seed)
    transmat = np.array([[0.9976, 0.0024],
                         [0.0232, 0.9768]])
    mu_dict  = {0: 0.0006,   1: -0.000881}
    sig_dict = {0: 0.00757, 1: 0.0163}

    base_corr = np.full((N_ASSETS, N_ASSETS), 0.185)
    np.fill_diagonal(base_corr, 1.0)

    all_states = np.zeros((num_days, N_ASSETS), dtype=int)
    for i in range(N_ASSETS):
        s = np.zeros(num_days, dtype=int)
        s[0] = np_rng.integers(2)
        for t in range(1, num_days):
            s[t] = np_rng.choice(2, p=transmat[s[t - 1]])
        all_states[:, i] = s

    rets = np.zeros((num_days, N_ASSETS))
    for t in range(num_days):
        mu_vec  = np.zeros(N_ASSETS)
        sig_vec = np.zeros(N_ASSETS)
        for i in range(N_ASSETS):
            curr_state = all_states[t, i]
            mu_vec[i]  = mu_dict[curr_state]
            sig_vec[i] = sig_dict[curr_state]
        cov_t = np.outer(sig_vec, sig_vec) * base_corr
        rets[t] = np_rng.multivariate_normal(mean=mu_vec, cov=cov_t)

    return pd.DataFrame(rets, columns=ASSETS), all_states


def simulate_3state_data(num_days, seed=None):
    np_rng = np.random.default_rng(seed)
    transmat = np.array([
        [0.9989, 0.0004, 0.0007],
        [0.0089, 0.9904, 0.0007],
        [0.0089, 0.0004, 0.9907]
    ])
    mu_list  = [0.0008, 0.0,     -0.003586]
    sig_list = [0.0070, 0.0050,  0.01897]

    base_corr = np.full((N_ASSETS, N_ASSETS), 0.185)
    np.fill_diagonal(base_corr, 1.0)

    all_states = np.zeros((num_days, N_ASSETS), dtype=int)
    for i in range(N_ASSETS):
        s = np.zeros(num_days, dtype=int)
        s[0] = np_rng.integers(3)
        for t in range(1, num_days):
            s[t] = np_rng.choice(3, p=transmat[s[t - 1]])
        all_states[:, i] = s

    rets = np.zeros((num_days, N_ASSETS))
    for t in range(num_days):
        mu_vec  = np.zeros(N_ASSETS)
        sig_vec = np.zeros(N_ASSETS)
        for i in range(N_ASSETS):
            st_i   = all_states[t, i]
            mu_vec[i]  = mu_list[st_i]
            sig_vec[i] = sig_list[st_i]
        cov_t = np.outer(sig_vec, sig_vec) * base_corr
        rets[t] = np_rng.multivariate_normal(mean=mu_vec, cov=cov_t)

    return pd.DataFrame(rets, columns=ASSETS), all_states

#### 3. Training Regime Models

#### 3.1 Hidden Markov Model

In [7]:
def run_mle(observations, n_components=2, init_type='default', seed=None):
    model = GaussianHMM(
        n_components=n_components,
        covariance_type='diag',
        n_iter=100,
        random_state=seed
    )
    if init_type == 'default':
        model.startprob_ = np.array([1.0, 0.0])
        model.transmat_  = np.array([
            [0.9, 0.1],
            [0.1, 0.9]
        ])
        model.means_  = np.zeros((n_components, observations.shape[1]))
        model.covars_ = np.full((n_components, observations.shape[1]), 1e-10)
        model.init_params = ''
    elif init_type == 'kmeans':
        km = KMeans(n_clusters=n_components, n_init=10, random_state=seed)
        labels = km.fit_predict(observations)
        means, covars = [], []
        for i in range(n_components):
            obs_i = observations[labels == i]
            means.append(np.mean(obs_i, axis=0))
            covars.append(np.var(obs_i, axis=0) + 1e-10)
        model.startprob_ = np.ones(n_components) / n_components
        model.transmat_  = np.ones((n_components, n_components)) / n_components
        model.means_     = np.array(means)
        model.covars_    = np.array(covars)
        model.init_params = ''

    model.fit(observations)
    pred_states = model.predict(observations)
    return model, pred_states


def run_mle_default(observations, seed=None):
    return run_mle(observations, init_type='default', seed=seed)


def run_mle_kmeans(observations, seed=None):
    return run_mle(observations, init_type='kmeans', seed=seed)


def train_hmm_single_asset_default(series, n_components=2, random_state=42):
    X = series.values.reshape(-1, 1)
    model, _ = run_mle_default(X, seed=random_state)
    return model


def train_hmm_single_asset_kmeans(series, n_components=2, random_state=42):
    X = series.values.reshape(-1, 1)
    model, _ = run_mle_kmeans(X, seed=random_state)
    return model

#### 3.2 Feature selection and SJM training

In [8]:
def compute_sjm_features(factor_ser: pd.Series) -> pd.DataFrame:
    def ewm_downside_dev(returns: pd.Series, halflife: float) -> pd.Series:
        negative_returns = returns.clip(upper=0)
        neg_sq = negative_returns ** 2
        ewm_mean = neg_sq.ewm(halflife=halflife, adjust=False).mean()
        return np.sqrt(ewm_mean)

    dd20 = ewm_downside_dev(factor_ser, halflife=20)
    dd60 = ewm_downside_dev(factor_ser, halflife=60)
    dd120 = ewm_downside_dev(factor_ser, halflife=120)

    feat_dd20 = dd20
    feat_dd20_minus_dd60 = dd20 - dd60
    feat_dd60_minus_dd120 = dd60 - dd120
    ret_hl120 = factor_ser.ewm(halflife=120, adjust=False).mean()

    feats = {
        "DD_hl20": feat_dd20,
        "DD20_minus_DD60": feat_dd20_minus_dd60,
        "DD60_minus_DD120": feat_dd60_minus_dd120,
        "Return_hl120": ret_hl120,
    }
    df_feats = pd.DataFrame(feats)
    df_feats = df_feats.replace([np.inf, -np.inf], np.nan).fillna(0.0)
    return df_feats

In [9]:
def train_sjm_single_asset(series, n_components=2, max_feats=4, lam=50, random_state=42):
    feats_df = compute_sjm_features(series)
    feats_df = feats_df.replace([np.inf, -np.inf], np.nan).fillna(0.0)

    clipper = DataClipperStd(mul=3.0)
    scaler  = StandardScalerPD()

    X_clipped = clipper.fit_transform(feats_df)
    X_scaled  = scaler.fit_transform(X_clipped)
    X_arr = X_scaled.values

    sjm = SparseJumpModel(
        n_components=n_components,
        max_feats=max_feats,
        jump_penalty=lam,
        cont=False,
        max_iter=20,
        random_state=random_state
    )
    sjm.fit(X_arr)

    return sjm, clipper, scaler

#### 4. Allocation simulation

#### 4.1 Backtest a static portfolio with single allocaiton

In [10]:
def backtest_portfolio(returns, weights, transaction_cost=0.0007):
    """
    Backtest a static portfolio with a single allocation across the entire
    test period. Includes an initial transaction cost.
    """
    T = len(returns)
    portfolio_vals = np.zeros(T)
    cost_init = np.sum(np.abs(weights)) * transaction_cost
    portfolio_vals[0] = 1.0 - cost_init

    for t in range(T - 1):
        ret_t = returns.iloc[t].values
        portfolio_vals[t + 1] = portfolio_vals[t] * (1.0 + np.dot(weights, ret_t))

    return portfolio_vals


#### 5.0 Performance Metric

In [11]:
def compute_performance_metrics(portfolio_vals, weight_history=None, annual_factor=250):
    pv = np.asarray(portfolio_vals)
    rets = np.diff(pv) / pv[:-1]

    ann_ret = rets.mean() * annual_factor
    cum_ret = pv[-1] / pv[0] - 1
    ann_vol = rets.std() * np.sqrt(annual_factor)
    negative_rets = rets[rets < 0]
    ddev = (negative_rets.std() * np.sqrt(annual_factor)) if len(negative_rets) > 0 else 0.0
    max_dd = (pv / np.maximum.accumulate(pv) - 1).min()

    sharpe = ann_ret / (ann_vol + 1e-12)
    sortino = ann_ret / ddev if ddev > 1e-12 else np.nan
    calmar  = ann_ret / abs(max_dd) if max_dd < 0 else np.nan

    if weight_history is not None and len(weight_history) > 1:
        turnovers = []
        for t in range(1, len(weight_history)):
            turnovers.append(np.sum(np.abs(weight_history[t] - weight_history[t-1])))
        avg_turnover = np.mean(turnovers)
    else:
        avg_turnover = 0.0

    return {
        "Annualized Return": ann_ret,
        "Cumulative Return": cum_ret,
        "Volatility": ann_vol,
        "Downside Deviation": ddev,
        "Max Drawdown": max_dd,
        "Sharpe Ratio": sharpe,
        "Sortino Ratio": sortino,
        "Calmar Ratio": calmar,
        "Turnover Rate": avg_turnover,
    }

#### 6. Helper Function: get per-regime means & std

In [12]:
def get_regime_means_stds_single_asset(asset_series, regime_assignments):
    """
    Returns two dicts:
      means_dict = {state: mean_return_in_that_state}
      stds_dict  = {state: std_return_in_that_state}
    """
    unique_states = np.unique(regime_assignments)
    regime_means = {}
    regime_stds  = {}
    for s in unique_states:
        data_in_s = asset_series[regime_assignments == s]
        if len(data_in_s) > 0:
            regime_means[s] = data_in_s.mean()
            regime_stds[s]  = data_in_s.std()
        else:
            # Fallback if empty
            regime_means[s] = asset_series.mean()
            regime_stds[s]  = asset_series.std()

    return regime_means, regime_stds

#### 7. Equal Unconditional Prior

In [13]:
def build_equal_unconditional_prior(df_train):
    """
    Build an 'equal unconditional prior' for BL using approximate known
    unconditional values for demonstration. 
    """
    SIM_MEAN_1STATE = 0.000461
    SIM_SIG_1STATE  = 0.008388
    TRUE_CORR       = 0.185

    n_assets = df_train.shape[1]
    uniform_corr = np.full((n_assets, n_assets), TRUE_CORR)
    np.fill_diagonal(uniform_corr, 1.0)

    cov_flat = (SIM_SIG_1STATE**2) * uniform_corr

    assets = df_train.columns
    pi_series = pd.Series(np.full(n_assets, SIM_MEAN_1STATE), index=assets)
    cov_df = pd.DataFrame(cov_flat, index=assets, columns=assets)
    return pi_series, cov_df

In [14]:
def get_rolling_cov(full_returns, current_index, halflife=126):
    """
    Compute an exponentially weighted covariance matrix using all data 
    available up to 'current_index', with an effective halflife equal to 'halflife' days.
    
    The equivalent span parameter is computed as:
        span_equiv = (2 / (1 - exp(-ln(2)/halflife))) - 1
    which ensures that the EWMA decay rate matches the specified halflife.
    
    If there are fewer than 2 data points, an identity matrix is returned.
    """
    # Use all available data from the beginning until current_index
    sub_df = full_returns.iloc[:current_index]
    
    if len(sub_df) < 2:
        n_ = sub_df.shape[1]
        return np.eye(n_)
    
    # Calculate the equivalent span for the desired halflife
    span_equiv = (2 / (1 - np.exp(-np.log(2) / halflife))) - 1
    
    # Compute the exponentially weighted covariance matrix
    cov_est = risk_models.exp_cov(sub_df, span=span_equiv, returns_data=True)
    return cov_est


#### 8.0 Regime-Based BL with the 'Equal Unconditional Prior'

In [15]:
def regime_based_bl_backtest_flatprior(
    df_train,
    df_test,
    states_test,
    init_state,  # initial state from training for each asset
    regime_means_list,
    transaction_cost=0.0007,
    risk_free_rate=0.02/252,
    bl_tau=0.05,
    halflife=126
):
    T_test = len(df_test)
    assets = df_test.columns
    n_assets = len(assets)
    
    # Combined data for rolling covariance and flat prior
    combined_df = pd.concat([df_train, df_test], axis=0).reset_index(drop=True)
    flat_pi, _ = build_equal_unconditional_prior(df_train)

    portfolio_vals = np.zeros(T_test)
    portfolio_vals[0] = 1.0
    weight_history = np.zeros((T_test, n_assets))
    w_prev = np.ones(n_assets) / n_assets
    weight_history[0] = w_prev

    # --- New: initialize fallback records list ---
    fallback_records = []

    for t in range(1, T_test):
        ret_t_minus_1 = df_test.iloc[t - 1].values
        gross_growth = portfolio_vals[t - 1] * (1.0 + np.dot(w_prev, ret_t_minus_1))
        
        if t == 1:
            current_states = init_state
            do_rebalance = True
        else:
            current_states = states_test[t - 1]
            do_rebalance = np.any(states_test[t - 1] != states_test[t - 2])
        
        if do_rebalance:
            view_vector = np.zeros(n_assets)
            for i in range(n_assets):
                view_vector[i] = regime_means_list[i].get(current_states[i], 0.0)
            
            global_index = len(df_train) + t
            cov_t = get_rolling_cov(combined_df, global_index, halflife=halflife)
            
            bl = BlackLittermanModel(
                cov_matrix=cov_t,
                pi="equal",
                absolute_views=dict(zip(assets, view_vector)),
                tau=bl_tau,
                risk_aversion=2.5
            )
            bl_rets = bl.bl_returns()

            ef = EfficientFrontier(bl_rets, cov_t, weight_bounds=(0, 1), solver="SCS")
            try:
                w_dict = ef.max_sharpe(risk_free_rate=risk_free_rate)
            except ValueError as e:
                print(f"Max Sharpe optimization failed at t = {t} with error: {e}")
                # --- Record fallback event details ---
                fallback_records.append({
                    "time": t,
                    "predicted_regime": current_states.copy(),
                    "absolute_views": dict(zip(assets, view_vector)),
                    "posterior_views": bl_rets.copy() if isinstance(bl_rets, np.ndarray) else bl_rets,
                })
                # Fallback to equal weights
                w_dict = {a: 1.0 / n_assets for a in assets}
            w_array = np.array([w_dict[a] for a in assets])
        else:
            w_array = w_prev.copy()

        traded_fraction = np.sum(np.abs(w_array - w_prev))
        cost = gross_growth * traded_fraction * transaction_cost
        portfolio_vals[t] = gross_growth - cost

        weight_history[t] = w_array
        w_prev = w_array

    return portfolio_vals, weight_history, fallback_records


#### 9. Wrapper to run all strategies

In [16]:
def equal_weight_allocation(n_assets):
    return np.ones(n_assets)/n_assets

def inverse_vol_weights(returns):
    stds = returns.std(axis=0).values + 1e-12
    w = 1.0/stds
    return w / w.sum()

def static_mvo_allocation(returns, risk_free_rate=0.02/252):
    mu = expected_returns.mean_historical_return(returns, frequency=250)
    raw_cov = risk_models.sample_cov(returns)
    ridge_lambda = 1e-5
    cov = raw_cov + np.eye(len(raw_cov)) * ridge_lambda
    try:
        ef = EfficientFrontier(mu, cov, weight_bounds=(0, 1), solver="SCS")
        w_dict = ef.max_sharpe(risk_free_rate=risk_free_rate)
    except ValueError as e:
        print(f"Falling back to min_volatility due to: {e}")
        ef = EfficientFrontier(mu, cov, weight_bounds=(0,1), solver="SCS")
        w_dict = ef.min_volatility()
    return ef.clean_weights()

In [17]:
def run_allocation(
    df,
    lam_sjm=50,
    risk_free_rate=0.02/252,
    transaction_cost=0.0007,
    bl_tau=0.05
):
    split_idx = int(len(df) * 0.8)
    df_train = df.iloc[:split_idx]
    df_test  = df.iloc[split_idx:]

    # Train per-asset models
    hmm_models_default = []
    hmm_models_kmeans  = []
    sjm_models         = []
    sjm_clippers       = []
    sjm_scalers        = []

    hmm_states_default_train = np.zeros((split_idx, N_ASSETS), dtype=int)
    hmm_states_kmeans_train  = np.zeros((split_idx, N_ASSETS), dtype=int)
    sjm_states_train         = np.zeros((split_idx, N_ASSETS), dtype=int)

    for i, asset in enumerate(ASSETS):
        series_train = df_train[asset]

        # HMM default
        hmm_d = train_hmm_single_asset_default(series_train)
        st_def = hmm_d.predict(series_train.values.reshape(-1, 1))
        hmm_models_default.append(hmm_d)
        hmm_states_default_train[:, i] = st_def

        # HMM kmeans
        hmm_k = train_hmm_single_asset_kmeans(series_train)
        st_km = hmm_k.predict(series_train.values.reshape(-1, 1))
        hmm_models_kmeans.append(hmm_k)
        hmm_states_kmeans_train[:, i] = st_km

        # SJM
        sjm_mod, sjm_clip, sjm_scale = train_sjm_single_asset(
            series_train, n_components=2, max_feats=4, lam=lam_sjm
        )
        feats_train = compute_sjm_features(series_train).replace([np.inf, -np.inf], np.nan).fillna(0.0)
        X_train_clip = sjm_clip.transform(feats_train)
        X_train_scl  = sjm_scale.transform(X_train_clip)
        st_sjm = sjm_mod.predict(X_train_scl)

        sjm_models.append(sjm_mod)
        sjm_clippers.append(sjm_clip)
        sjm_scalers.append(sjm_scale)
        sjm_states_train[:, i] = st_sjm

    # In-sample regime means
    hmm_regime_means_default = []
    hmm_regime_means_kmeans  = []
    sjm_regime_means         = []

    for i in range(N_ASSETS):
        asset_train = df_train.iloc[:, i]

        # HMM default
        m_def, _ = get_regime_means_stds_single_asset(asset_train, hmm_states_default_train[:, i])
        hmm_regime_means_default.append(m_def)

        # HMM kmeans
        m_km, _ = get_regime_means_stds_single_asset(asset_train, hmm_states_kmeans_train[:, i])
        hmm_regime_means_kmeans.append(m_km)

        # SJM
        m_sjm, _ = get_regime_means_stds_single_asset(asset_train, sjm_states_train[:, i])
        sjm_regime_means.append(m_sjm)

    # Predict states on the test day-by-day
    T_test = len(df_test)
    hmm_states_default_test = np.zeros((T_test, N_ASSETS), dtype=int)
    hmm_states_kmeans_test  = np.zeros((T_test, N_ASSETS), dtype=int)
    sjm_states_test         = np.zeros((T_test, N_ASSETS), dtype=int)

    # Full series for partial inference
    full_series = pd.concat([df_train, df_test], axis=0).reset_index(drop=True)

    for i, asset in enumerate(ASSETS):
        asset_full = full_series[asset].values

        # HMM default
        for t in range(T_test):
            end_idx = split_idx + t + 1
            partial_data = asset_full[:end_idx].reshape(-1, 1)
            partial_states = hmm_models_default[i].predict(partial_data)
            hmm_states_default_test[t, i] = np.asarray(partial_states)[-1]

        # HMM kmeans
        for t in range(T_test):
            end_idx = split_idx + t + 1
            partial_data = asset_full[:end_idx].reshape(-1, 1)
            partial_states = hmm_models_kmeans[i].predict(partial_data)
            hmm_states_kmeans_test[t, i] = np.asarray(partial_states)[-1]

        # SJM
        feats_full = compute_sjm_features(pd.Series(asset_full))
        feats_full = feats_full.replace([np.inf, -np.inf], np.nan).fillna(0.0)
        X_full_clip = sjm_clippers[i].transform(feats_full)
        X_full_scl  = sjm_scalers[i].transform(X_full_clip)
        for t in range(T_test):
            end_idx = split_idx + t + 1
            partial_X = X_full_scl[:end_idx]
            partial_states = sjm_models[i].predict(partial_X)
            sjm_states_test[t, i] = np.asarray(partial_states)[-1]

    # Evaluate Strategies

    # A) Equal Weight
    w_ew = equal_weight_allocation(N_ASSETS)
    pv_ew = backtest_portfolio(df_test, w_ew, transaction_cost=transaction_cost)
    w_hist_ew = np.tile(w_ew, (T_test, 1))

    # B) Inverse Vol
    w_iv = inverse_vol_weights(df_test)
    pv_iv = backtest_portfolio(df_test, w_iv, transaction_cost=transaction_cost)
    w_hist_iv = np.tile(w_iv, (T_test, 1))

    # C) Static MVO (trained on df_train)
    w_mvo_dict = static_mvo_allocation(df_train, risk_free_rate=risk_free_rate)
    w_mvo_arr = np.array([w_mvo_dict[a] for a in df_train.columns])
    pv_mvo = backtest_portfolio(df_test, w_mvo_arr, transaction_cost=transaction_cost)
    w_hist_mvo = np.tile(w_mvo_arr, (T_test, 1))

    # D) HMM-BL (Default)
    pv_hmmbl_def, w_hmmbl_def, fallback_hmmbl_def = regime_based_bl_backtest_flatprior(
        df_train,
        df_test,
        hmm_states_default_test,
        hmm_states_default_train[-1, :],  # Use last training state for initialization
        hmm_regime_means_default,
        transaction_cost=transaction_cost,
        risk_free_rate=risk_free_rate,
        bl_tau=bl_tau,
        halflife=126
    )

    # E) HMM-BL (KMeans)
    pv_hmmbl_km, w_hmmbl_km, fallback_hmmbl_km = regime_based_bl_backtest_flatprior(
        df_train,
        df_test,
        hmm_states_kmeans_test,
        hmm_states_kmeans_train[-1, :],  # Use last training state for initialization
        hmm_regime_means_kmeans,
        transaction_cost=transaction_cost,
        risk_free_rate=risk_free_rate,
        bl_tau=bl_tau,
        halflife=126
    )

    # F) SJM-BL
    pv_sjmbl, w_sjmbl, fallback_sjmbl = regime_based_bl_backtest_flatprior(
        df_train,
        df_test,
        sjm_states_test,
        sjm_states_train[-1, :],  # Use last training state for initialization
        sjm_regime_means,
        transaction_cost=transaction_cost,
        risk_free_rate=risk_free_rate,
        bl_tau=bl_tau,
        halflife=126
    )

    perf = {
        "EW": compute_performance_metrics(pv_ew, w_hist_ew),
        "IV": compute_performance_metrics(pv_iv, w_hist_iv),
        "MVO": compute_performance_metrics(pv_mvo, w_hist_mvo),
        "HMM-BL-Default": compute_performance_metrics(pv_hmmbl_def, w_hmmbl_def),
        "HMM-BL-KMeans":  compute_performance_metrics(pv_hmmbl_km, w_hmmbl_km),
        "SJM-BL":         compute_performance_metrics(pv_sjmbl, w_sjmbl)
    }

    # Bundle fallback events from the regime-based strategies
    fallback_events = {
        "HMM-BL-Default": fallback_hmmbl_def,
        "HMM-BL-KMeans": fallback_hmmbl_km,
        "SJM-BL": fallback_sjmbl
    }

    return perf, fallback_events

#### 10. FUll scenario 1-state, 2-state, 3-state

In [18]:
def run_scenario_123(
    T_sim=1000,
    lam_sjm=50,
    risk_free_rate=0.02/252,
    transaction_cost=0.0007,
    bl_tau=0.05,
    seed1=None,
    seed2=None,
    seed3=None
):
    df1_full = simulate_1state_data(T_sim, seed=seed1)
    perf_1 = run_allocation(
        df1_full,
        lam_sjm=lam_sjm,
        risk_free_rate=risk_free_rate,
        transaction_cost=transaction_cost,
        bl_tau=bl_tau
    )

    df2_full, _ = simulate_2state_data(T_sim, seed=seed2)
    perf_2 = run_allocation(
        df2_full,
        lam_sjm=lam_sjm,
        risk_free_rate=risk_free_rate,
        transaction_cost=transaction_cost,
        bl_tau=bl_tau
    )

    df3_full, _ = simulate_3state_data(T_sim, seed=seed3)
    perf_3 = run_allocation(
        df3_full,
        lam_sjm=lam_sjm,
        risk_free_rate=risk_free_rate,
        transaction_cost=transaction_cost,
        bl_tau=bl_tau
    )

    return {
        "1state": perf_1,
        "2state": perf_2,
        "3state": perf_3
    }


def single_monte_carlo_run(
    run_id,
    T_sim=1000,
    lam_sjm=50,
    risk_free_rate=0.02/252,
    transaction_cost=0.0007,
    bl_tau=0.05
):
    print(f"Running simulation {run_id}...")
    seed_for_1state = run_id * 1000 + 11
    seed_for_2state = run_id * 1000 + 22
    seed_for_3state = run_id * 1000 + 33

    results = run_scenario_123(
        T_sim=T_sim,
        lam_sjm=lam_sjm,
        risk_free_rate=risk_free_rate,
        transaction_cost=transaction_cost,
        bl_tau=bl_tau,
        seed1=seed_for_1state,
        seed2=seed_for_2state,
        seed3=seed_for_3state
    )
    return results


def run_monte_carlo_study(
    n_runs=10,
    T_sim=1000,
    lam_sjm=50,
    risk_free_rate=0.02/252,
    transaction_cost=0.0007,
    bl_tau=0.05
):
    print(f"Detected {multiprocessing.cpu_count()} cores")
    all_results = Parallel(n_jobs=-1)(
        delayed(single_monte_carlo_run)(
            i+1, T_sim, lam_sjm, risk_free_rate, transaction_cost, bl_tau
        )
        for i in range(n_runs)
    )

    strategies = ["EW", "IV", "MVO", "HMM-BL-Default", "HMM-BL-KMeans", "SJM-BL"]
    scenarios  = ["1state", "2state", "3state"]

    sharpe_data = {sc: {st: [] for st in strategies} for sc in scenarios}
    all_metrics = {sc: {} for sc in scenarios}
    for sc in scenarios:
        all_metrics[sc] = {}
        for st in strategies:
            all_metrics[sc][st] = {
                "Annualized Return": [],
                "Cumulative Return": [],
                "Volatility": [],
                "Downside Deviation": [],
                "Max Drawdown": [],
                "Sharpe Ratio": [],
                "Sortino Ratio": [],
                "Calmar Ratio": [],
                "Turnover Rate": [],
            }

    # Gather distribution of metrics
    for run_res in all_results:
        for sc in scenarios:
            # Unpack the tuple (perf, fallback_events)
            perf, fallback_events = run_res[sc]
            for st in strategies:
                metrics_dict = perf[st]
                sharpe_data[sc][st].append(metrics_dict["Sharpe Ratio"])
                for mkey in all_metrics[sc][st]:
                    all_metrics[sc][st][mkey].append(metrics_dict[mkey])

    # Wilcoxon tests
    print("\n==== Wilcoxon Tests (SJM-BL vs. others) ====")
    wilcoxon_rows = []
    for sc in scenarios:
        sjm_sharpes = sharpe_data[sc]["SJM-BL"]
        for st in strategies:
            if st == "SJM-BL":
                continue
            other_sharpes = sharpe_data[sc][st]
            try:
                stat, pval = wilcoxon(sjm_sharpes, other_sharpes, alternative='two-sided')
            except ValueError:
                stat, pval = np.nan, np.nan
            print(f"{sc} | SJM-BL vs {st}: stat={stat:.4f}, p={pval:.4g}")
            wilcoxon_rows.append({
                "Scenario": sc,
                "Comparison": f"SJM-BL vs {st}",
                "Statistic": stat,
                "p-value": pval
            })

    df_wilcoxon = pd.DataFrame(wilcoxon_rows)
    print("\nWilcoxon Results Table:")
    print(df_wilcoxon.to_string(index=False))

    # Print average metrics
    print("\n==== Average Performance Metrics (across runs) ====")
    for sc in scenarios:
        rows = []
        for st in strategies:
            metric_means = {}
            for mkey, vals in all_metrics[sc][st].items():
                metric_means[mkey] = np.mean(vals)
            row = {"Strategy": st}
            row.update(metric_means)
            rows.append(row)
        df_avg = pd.DataFrame(rows)
        df_avg.set_index("Strategy", inplace=True)
        print(f"\n--- {sc.upper()} ---")
        print(df_avg.to_string())

    return sharpe_data, all_metrics, all_results, df_wilcoxon

#### Main Exectution

In [19]:
if __name__ == "__main__":
    config = {
        "n_simulations": 500,
        "T_sim": 5000,
        "risk_free_rate": 0.02 / 252,
        "transaction_cost": 0.0007,  # 7 bps
        "bl_tau": 0.05,
        "lam_sjm": 100,
    }

    sharpe_data, all_metrics, all_runs, df_wilcoxon = run_monte_carlo_study(
        n_runs=config["n_simulations"],
        T_sim=config["T_sim"],
        lam_sjm=config["lam_sjm"],
        risk_free_rate=config["risk_free_rate"],
        transaction_cost=config["transaction_cost"],
        bl_tau=config["bl_tau"]
    )

    # Fallback summary loop now uses all_runs
    fallback_summary = []
    for run in all_runs:
        for scenario in run:  # "1state", "2state", "3state"
            # Unpack the tuple to get fallback_events (second element)
            _, fallback_dict = run[scenario]
            for strat, events in fallback_dict.items():
                for event in events:
                    record = event.copy()
                    record.update({
                        "Scenario": scenario,
                        "Strategy": strat
                    })
                    fallback_summary.append(record)
    if fallback_summary:
        df_fallback = pd.DataFrame(fallback_summary)
        print("\n==== Fallback Events Summary ====")
        print(df_fallback.to_string(index=False))
    else:
        print("\nNo fallback events recorded.")

Detected 8 cores


Running simulation 8...
Running simulation 2...
Running simulation 7...
Running simulation 6...
Running simulation 5...
Running simulation 4...
Running simulation 3...
Running simulation 1...


Model is not converging.  Current: 13461.444512960772 is not greater than 13461.462574048224. Delta is -0.018061087452224456
Model is not converging.  Current: 13399.97468682194 is not greater than 13400.026712611634. Delta is -0.05202578969328897
Model is not converging.  Current: 13430.843406489204 is not greater than 13430.876458913983. Delta is -0.033052424778361456
Model is not converging.  Current: 13417.123647420682 is not greater than 13417.135908083997. Delta is -0.012260663315828424
Model is not converging.  Current: 13477.861014403064 is not greater than 13477.897984230327. Delta is -0.03696982726251008
Model is not converging.  Current: 13458.44335717629 is not greater than 13458.466881097735. Delta is -0.02352392144530313
Model is not converging.  Current: 13384.283015944642 is not greater than 13384.291657760492. Delta is -0.008641815849841805
Model is not converging.  Current: 13384.939811275972 is not greater than 13384.963567917232. Delta is -0.023756641259751632
Model

Error in LDL factorization when computing the nonzero elements. The problem seems to be non-convex.
factor_status: 0, num_vars: 7
Error in LDL initial factorization.
ERROR: init_lin_sys_work failure
Falling back to min_volatility due to: ScsWork allocation error!
Error in LDL factorization when computing the nonzero elements. The problem seems to be non-convex.
factor_status: 0, num_vars: 7
Error in LDL initial factorization.
ERROR: init_lin_sys_work failure
Falling back to min_volatility due to: ScsWork allocation error!
Error in LDL factorization when computing the nonzero elements. The problem seems to be non-convex.
factor_status: 0, num_vars: 7
Error in LDL initial factorization.
ERROR: init_lin_sys_work failure
Falling back to min_volatility due to: ScsWork allocation error!
Error in LDL factorization when computing the nonzero elements. The problem seems to be non-convex.
factor_status: 0, num_vars: 7
Error in LDL initial factorization.
ERROR: init_lin_sys_work failure
Falling b

Model is not converging.  Current: 13516.67396583896 is not greater than 13516.680757591486. Delta is -0.006791752526623895
Model is not converging.  Current: 13519.865165012987 is not greater than 13519.95135763685. Delta is -0.08619262386309856
Model is not converging.  Current: 13556.557441620107 is not greater than 13556.55910009154. Delta is -0.0016584714321652427
Model is not converging.  Current: 13556.552089422566 is not greater than 13556.556328213921. Delta is -0.004238791354509885
Model is not converging.  Current: 13322.759857633637 is not greater than 13322.791455116407. Delta is -0.031597482769939234
Model is not converging.  Current: 13322.75963222329 is not greater than 13322.787329813167. Delta is -0.02769758987778914
Model is not converging.  Current: 13531.08704238805 is not greater than 13531.122596417064. Delta is -0.03555402901474736
Model is not converging.  Current: 13475.803503442388 is not greater than 13475.807757561965. Delta is -0.004254119576216908
Model i

...

Error in LDL factorization when computing the nonzero elements. The problem seems to be non-convex.
factor_status: 0, num_vars: 7
Error in LDL initial factorization.
ERROR: init_lin_sys_work failure
Falling back to min_volatility due to: ScsWork allocation error!


Model is not converging.  Current: 14092.803884566922 is not greater than 14092.81634623506. Delta is -0.012461668138712412
Model is not converging.  Current: 14094.990796971855 is not greater than 14095.008871751772. Delta is -0.01807477991678752
Model is not converging.  Current: 13324.333859035365 is not greater than 13324.348980620905. Delta is -0.015121585540327942
Model is not converging.  Current: 13556.494632869053 is not greater than 13556.55315919692. Delta is -0.058526327866275096
Model is not converging.  Current: 13556.450504664177 is not greater than 13556.452895640357. Delta is -0.0023909761803224683
Model is not converging.  Current: 13424.437329485934 is not greater than 13424.503368217798. Delta is -0.06603873186395504


Error in LDL factorization when computing the nonzero elements. The problem seems to be non-convex.
factor_status: 0, num_vars: 7
Error in LDL initial factorization.
ERROR: init_lin_sys_work failure
Falling back to min_volatility due to: ScsWork allocation error!
Error in LDL factorization when computing the nonzero elements. The problem seems to be non-convex.
factor_status: 0, num_vars: 7
Error in LDL initial factorization.
ERROR: init_lin_sys_work failure
Falling back to min_volatility due to: ScsWork allocation error!
Error in LDL factorization when computing the nonzero elements. The problem seems to be non-convex.
factor_status: 0, num_vars: 7
Error in LDL initial factorization.
ERROR: init_lin_sys_work failure
Falling back to min_volatility due to: ScsWork allocation error!
Error in LDL factorization when computing the nonzero elements. The problem seems to be non-convex.
factor_status: 0, num_vars: 7
Error in LDL initial factorization.
ERROR: init_lin_sys_work failure
Falling b

Model is not converging.  Current: 14038.806957724548 is not greater than 14038.830630058597. Delta is -0.023672334049479105
Model is not converging.  Current: 14031.081755576146 is not greater than 14031.121313826035. Delta is -0.03955824988952372
Model is not converging.  Current: 14031.082043622167 is not greater than 14031.101001543677. Delta is -0.01895792150935449
Model is not converging.  Current: 13518.75052077057 is not greater than 13518.751636809577. Delta is -0.0011160390058648773
Model is not converging.  Current: 13944.795042001384 is not greater than 13944.810905870843. Delta is -0.015863869459280977
Model is not converging.  Current: 13944.794744217168 is not greater than 13944.807991298012. Delta is -0.01324708084393933


Error in LDL factorization when computing the nonzero elements. The problem seems to be non-convex.
factor_status: 0, num_vars: 7
Error in LDL initial factorization.
ERROR: init_lin_sys_work failure
Falling back to min_volatility due to: ScsWork allocation error!

==== Wilcoxon Tests (SJM-BL vs. others) ====
1state | SJM-BL vs EW: stat=43640.0000, p=4.267e-09
1state | SJM-BL vs IV: stat=42729.0000, p=7.493e-10
1state | SJM-BL vs MVO: stat=17798.0000, p=9.853e-44
1state | SJM-BL vs HMM-BL-Default: stat=57400.0000, p=0.106
1state | SJM-BL vs HMM-BL-KMeans: stat=77.0000, p=2.008e-83
2state | SJM-BL vs EW: stat=47741.0000, p=4.13e-06
2state | SJM-BL vs IV: stat=62355.0000, p=0.9334
2state | SJM-BL vs MVO: stat=17634.0000, p=4.851e-44
2state | SJM-BL vs HMM-BL-Default: stat=57563.0000, p=0.1173
2state | SJM-BL vs HMM-BL-KMeans: stat=30505.0000, p=2.87e-23
3state | SJM-BL vs EW: stat=11605.0000, p=3.991e-56
3state | SJM-BL vs IV: stat=30102.0000, p=8.15e-24
3state | SJM-BL vs MVO: stat=9925.

In [20]:
import pickle
import pandas as pd

# Save non-DataFrame objects using pickle
results = {
    "sharpe_data": sharpe_data,
    "all_metrics": all_metrics,
    "all_runs": all_runs,
    "df_wilcoxon": df_wilcoxon
}

with open("simulation_results.pkl", "wb") as f:
    pickle.dump(results, f)

# Save fallback summary DataFrame to a CSV file, if available
if fallback_summary:
    df_fallback.to_csv("fallback_events_summary.csv", index=False)
    
# Optionally, if df_wilcoxon is a DataFrame, you can also save it as CSV
if isinstance(df_wilcoxon, pd.DataFrame):
    df_wilcoxon.to_csv("wilcoxon_results.csv", index=False)

print("All simulation data has been saved.")


All simulation data has been saved.
